# 다익스트라 알고리즘

특정 노드에서 다른 모든 노드로 가는 최단 경로 탐색 알고리즘.
음의 간선이 없을 때 정상적으로 작동.
그리디 알고리즘

1. 출발 노드 설정
2. 최단 거리 테이블 초기화 (자기 자신 노드는 0, 나머지는 무한대로 설정)
3. 방문하지 않은 노드 중에서 가장 짧은 노드 선택(그리디)
4. 비용 계산하여 테이블 갱신
5. 3,4과정 반복


In [2]:
INF = int(1e9) # 10억

#노드와 간선의 개수 입력 받기
n, m = map(int,input().split())
#시작 노드 입력 받기
start = int(input())
#각 노드에 연결되어 있는 노드 정보를 담는 리스트 만들기
graph = [[] for i in range(n+1)] # 노드 번호와 index를 동일하게 하기 위해 n+1
#방문 체크 리스트 만들기
visited = [False] * (n+1)
#최단 거리 테이블 무한대로 초기화
distance = [INF] * (n+1)

#간선 정보 입력
for _ in range(m):
    a,b,c = map(int,input().split())
    graph[a].append((b,c)) # a번 노드에서 b번 노드로 가는 비용이 c

#방문하지 않은 노드중에서 가장 거리가 짧은 노드 번호 반환
def get_smallest_node():
    min_value = INF
    index = 0 #가장 최단 거리가 짧은 노드(인덱스)
    for i in range(1,n+1):
        if distance[i]<min_value and not visited[i]:
            min_value = distance[i]
            index = i
    return index

def dijkstra(start):
    #시작 노드에 대해 초기화
    distance[start] = 0
    visited[start] = True
    for j in graph[start]:
        distance[j[0]] = j[1]
    #시작 노드를 제외한 전체 n-1에 대해 반복
    for i in range(n-1):
        #현재 최단 거리가 가장 짧은 노드에 대해 방문 처리
        now = get_smallest_node()
        visited[now] = True
        #현재 노드와 연결된 다른 노드 확인
        for j in graph[now]:
            cost = distance[now] + j[1]
            #현재 노드를 거쳐서 다른 노드로 가는 거리가 더 짧은 경우
            if cost < distance[j[0]]:
                distance[j[0]] = cost
                
#다익스트라 알고리즘 수행
dijkstra(start)

#모든 노드로 가기 위한 최단거리 출력
for i in range(1,n+1):
    #도달할 수 없는 경우
    if distance[i]==INF:
        print('INFINITY')
    #도달할 수 있는 경우 거리를 출력 
    else:
        print(distance[i])

6 11
1
1 2 2
1 4 1
1 3 5
2 4 2
2 3 3
3 2 3
3 6 5
4 3 3
4 5 1
5 3 1
5 6 2
0
2
3
1
2
4


위의 경우 시간 복잡도 O(n^2) 따라서 노드의 개수 많아지면 시간 초과

## 우선 순위 큐
우선 순위가 높은 순서대로 출력. 힙(heap)을 통해서 구현하면 시간복잡도 O(log n)

In [3]:
#Min_Heap
import heapq
#오름차순 힙정렬 (기본적으로 힙은 오름차순으로 넣음)
def heapsort(iterable):
    h=[]
    result=[]
    #모든 원소들을 차례대로 힙에 삽입
    for value in iterable:
        heapq.heappush(h,value)
    #힙에 삽입된 원소를 꺼내어 result에 담기
    for i in range(len(h)):
        result.append(heapq.heappop(h))
    return result

result = heapsort([4,2,7,9,0,1,6,3,5,8]) #시간복잡도 O(nlog n)
print(result)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [4]:
#Max_Heap
import heapq
#내림차순 힙정렬(부호바꿔서 오름차순으로 넣고 꺼낼때 다시 부호 바꿔줌)
def heapsort(iterable):
    h=[]
    result=[]
    for value in iterable:
        heapq.heappush(h,-value)
    for i in range(len(h)):
        result.append(-heapq.heappop(h))
    return result

result = heapsort([4,2,7,9,0,1,6,3,5,8]) #시간복잡도 O(nlog n)
print(result)

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


## 개선된 다익스트라 알고리즘
    우선 순위 큐에 넣을 때는 갱신된 노드의 정보만 넣음(distance[]원래의 거리와 이전 노드의 distance+다음노드의 graph[]두번째 원소값 비교)
    최단거리가 가장 짧은 노드를 선택할 때 힙 자료구조를 이용 -> 짧은 최단거리 노드를 구해야하므로 min_heap 사용

In [2]:
import heapq
INF = int(1e9)

n, m = map(int,input().split())
start = int(input())
graph = [[] for i in range(n+1)]
#visited 리스트 안쓰고 다익스트라 함수내에서 우선순위 큐에서 pop 된 것은 이미 방문한 것으로 처리
distance = [INF] * (n+1)


for _ in range(m):
    a,b,c = map(int,input().split())
    graph[a].append((b,c))

def dijkstra(start):
    q=[] #우선 순위 큐
    heapq.heappush(q,(0,start)) #거리0, 노드1을 차례대로 삽입.
    distance[start] = 0
    while q: #큐가 비어있지 않다면
        #가장 최단거리 짧은 노드 정보 꺼내기
        dist, now = heapq.heappop(q) 
        #현재 노드가 이미 처리된거라면 무시
        if distance[now] < dist: #현재 우선순위 큐에서 꺼낸 거리(dist)가 distance 테이블의 거리보다 크다면
            continue
        #현재 노드와 연결된 다른 인접한 노드들을 확인
        for i in graph[now]:
            cost = dist + i[1]
            #현재 노드를 거쳐서 다른 노드로 이동하는 거리가 더 짧은 경우
            if cost < distance[i[0]]:
                distance[i[0]] = cost #갱신된 거리를 distance 테이블에 기록
                heapq.heappush(q,(cost,i[0]))

#다익스트라 알고리즘 수행
dijkstra(start)

#모든 노드로 가기 위한 최단거리 출력
for i in range(1,n+1):
    #도달할 수 없는 경우
    if distance[i]==INF:
        print('INFINITY')
    #도달할 수 있는 경우 거리를 출력 
    else:
        print(distance[i])

6 11
1
1 2 2
1 4 1
1 3 5
2 4 2
2 3 3
3 2 3
3 6 5
4 3 3
4 5 1
5 3 1
5 6 2
0
2
3
1
2
4


#### 시간복잡도: O(ElogV)
    E: 간선의 개수
    V: 노드의 개수

# 플로이드 워셜 알고리즘
    다익스트라 알고리즘을 다이나믹 프로그램으로 처리(2차원 테이블에 최단거리 정보 저장)
    코드는 간단하나, 시간복잡도가 O(n^3) 이기 때문에 노드와 간선의 개수가 적을 때만 사용
    각 단계마다 노드 k를 거쳐가는 경우를 확인: 점화식 D ab = min(D ab, D ak + D kb)

In [6]:
INF = int(1e9) #무한대
#노드와 간선 개수 입력
n, m = map(int,input().split())
#2차원 테이블 무한대로 초기화
graph = [[INF]*(n+1) for _ in range(n+1)]
#아이덴티티 행렬 0으로 (자기자신의 거리 0으로)
for i in range(1,n+1):
    for j in range(1,n+1):
        if i==j:
            graph[i][j] = 0
#각 간선 정보 입력 (a에서 b까지 거리 c)
for i in range(m):
    a,b,c = map(int,input().split())
    graph[a][b] = c
#플로이드 워셜 알고리즘
for k in range(1,n+1):
    for i in range(1,n+1):
        for j in range(1,n+1):
            graph[i][j] = min(graph[i][j], graph[i][k]+graph[k][j])
#결과 출력
for i in range(1,n+1):
    for j in range(1,n+1):
        if graph[i][j] == 'INF':
            print('INFINITY')
        else:
            print(graph[i][j], end=' ')
    print()

4 7
1 2 4
1 4 6
2 1 3
2 3 7
3 1 5
3 4 4
4 3 2
0 4 8 6 
3 0 7 9 
5 9 0 4 
7 11 2 0 


## 문제 1 : 메시지를 보낼 수 있는 도시의 수와 걸리는 시간 출력

입력 첫 줄에는 도시의 수, 통로의 수, 보내는 도시 넘버 입력. 그 밑의 줄들은 보내는 도시, 받는 도시, 걸리는 시간을 입력
-> 메시지를 받을 수 있는 도시를 출력. 걸리는 총 시간을 출력해야하므로 가장 거리가 먼 도시를 출력하면 됨.

-> 한 도시에서 다른 도시로 가는 최단거리문제. 다익스트라 알고리즘 & 우선순위 큐 활용

In [3]:
#다익스트라 알고리즘: start 노드 우선 순위 큐에 넣는다.
    #큐를 pop 시킨 dist, 노드 번호에 대해 다음 노드로 가는 거리가 원래 그 노드로 가는 거리보다 작다면 distance 테이블을 교체
    #우선순위 큐에 넣는다. 반복.
import heapq
INF = int(1e9)

def dijkstra(start):
    q= [] #우선순위 큐
    distance[start]=0
    heapq.heappush(q,(0,start)) #거리와 노드번호 push
    while q:
        dist, now = heapq.heappop(q)
        if distance[now] < dist:
            continue
        for i in graph[now]:
            cost = dist + i[1]
            if cost < distance[i[0]]:
                distance[i[0]] = cost
                heapq.heappush(q,(cost,i[0]))
            
#도시의 개수와 간선, 시작 노드 번호 입력
n, m, start = map(int,input().split())
graph = [[] for i in range(n+1)]
distance = [INF] * (n+1)
for i in range(m): #x번 노드에서 y번 노드까지 걸리는 거리 z
    x, y, z = map(int,input().split())
    graph[x].append((y,z))

dijkstra(start)

count = 0 #도시의 개수 세기 위함
max_distance = 0

for d in distance:
    if d != INF:
        count += 1
        max_distance = max(d,max_distance)
        
print(count-1, max_distance) #시작 도시 제외
    

3 2 1
1 2 4
1 3 5
2 5


## 문제 2: 목표로 가기 위해 중간에 특정 노드는 거쳐야하며 간선은 양방향으로 갈 수 있다. 최소 이동 거리 구하시오. (도달할 수 없으면 -1)
    

    첫째 줄: 노드의 개수, 간선의 개수
    둘째 줄부터: 연결된 노드의 번호 2개 (공백으로 구분)
    마지막 줄: 목표 노드와 중간 특정 노드 (공백으로 구분)

In [4]:
#n의 크기가 최대 100까지 입력 가능하므로 플로이드 워셜 알고리즘 사용
#시작노드부터 특정노드의 거리 + 특정 노드부터 목표노드까지의 거리

In [8]:
INF = int(1e9) #무한

n, m = map(int,input().split()) #노드와 간선
graph = [[INF]*(n+1) for _ in range(n+1)] #2차원 리스트 테이블, INF로 초기화

#아이덴티티 matrix
for a in range(1,n+1):
    for b in range(1,n+1):
        if a == b:
            graph[a][b] = 0
#회사to회사 입력     
for _ in range(m):
    a, b = map(int,input().split())
    graph[a][b] = 1
    graph[b][a] = 1

#목표노드, 특정노드
x, k = map(int,input().split())

#플로이드 워셜
for k in range(1,n+1):
    for a in range(1,n+1):
        for b in range(1,n+1):
            graph[a][b] = min(graph[a][b], graph[a][k] + graph[k][b])
#최단거리            
distance = graph[1][k]+ graph[k][x]

if distance == INF:
    print(-1)
else:
    print(distance)

5 7
1 2
1 3
1 4
2 4
3 4
3 5
4 5
4 5
3
